In [1]:
import os
import json
import cv2
import multiprocessing as mp
from pathlib import Path
from tqdm import tqdm
import mediapipe as mp_pose_module
import subprocess

In [9]:
import os
from moviepy.editor import VideoFileClip
from tqdm import tqdm

input_folder = "/home/siamai/data/Penguin/week7/test"
output_folder = "/home/siamai/data/Penguin/week7/test"

# Get a list of all .mov files in the input folder
mov_files = [filename for filename in os.listdir(input_folder) if filename.endswith(".mov")]

# Loop through all .mov files and show estimated progress with a progress bar
for filename in tqdm(mov_files, desc='Converting videos', unit='video'):
    input_file_path = os.path.join(input_folder, filename)

    try:
        # Load the .mov file
        video = VideoFileClip(input_file_path).set_fps(24)

        # Safely get fps
        fps = video.fps if video.fps else 24

        # Check video properties
        if video.duration is None or video.size is None:
            print(f"❌ Skipping {filename} — missing duration or size")
            continue

        # Construct output file path
        output_filename = os.path.splitext(filename)[0] + ".mp4"
        output_file_path = os.path.join(output_folder, output_filename)

        print(f"\n🎬 Processing: {filename}")
        print("   video.fps:", fps)
        print("   video.duration:", video.duration)
        print("   video.size:", video.size)

        # Write the video to output
        video.write_videofile(output_file_path, codec='libx264', fps=fps)

        # Manual frame progress bar (optional)
        total_frames = int(video.duration * fps)
        progress = 0
        with tqdm(total=total_frames, desc=f'{filename}', unit='frame') as pbar:
            while progress < total_frames:
                progress = video.reader.nframes
                pbar.update(progress - pbar.n)

        video.close()

    except Exception as e:
        print(f"⚠️ Failed to convert {filename}: {e}")

Converting videos:   0%|          | 0/1 [00:00<?, ?video/s]


🎬 Processing: copy_B52269A0-899E-43A0-A841-93F6DBDD7A9F.mov
   video.fps: 24
   video.duration: 1861.82
   video.size: [2560, 1440]
Moviepy - Building video /home/siamai/data/Penguin/week7/test/copy_B52269A0-899E-43A0-A841-93F6DBDD7A9F.mp4.
MoviePy - Writing audio in copy_B52269A0-899E-43A0-A841-93F6DBDD7A9FTEMP_MPY_wvf_snd.mp3


Converting videos: 100%|██████████| 1/1 [00:13<00:00, 13.82s/video]

MoviePy - Done.
Moviepy - Writing video /home/siamai/data/Penguin/week7/test/copy_B52269A0-899E-43A0-A841-93F6DBDD7A9F.mp4

⚠️ Failed to convert copy_B52269A0-899E-43A0-A841-93F6DBDD7A9F.mov: must be real number, not NoneType


In [15]:
import os
from moviepy.editor import VideoFileClip
from tqdm import tqdm

input_folder = "/home/siamai/data/Penguin/week7/test"
output_folder = "/home/siamai/data/Penguin/week7/test"

mov_files = [f for f in os.listdir(input_folder) if f.endswith(".mov")]

for filename in tqdm(mov_files, desc='Converting videos', unit='video'):
    try:
        print(f"\n🎬 Processing: {filename}")
        input_file_path = os.path.join(input_folder, filename)
        output_filename = os.path.splitext(filename)[0] + ".mp4"
        output_file_path = os.path.join(output_folder, output_filename)

        video = VideoFileClip(input_file_path)
        fps = video.fps or 24
        duration = video.duration or 0
        size = video.size

        print("   video.fps:", fps)
        print("   video.duration:", duration)
        print("   video.size:", size)
        print("   video.has_audio:", video.audio is not None)

        # Set fps explicitly
        video = video.set_fps(fps)

        # Check and validate audio
        has_audio = False
        if video.audio is not None:
            try:
                _ = video.audio.fps
                _ = video.audio.duration
                _ = video.audio.nchannels
                has_audio = True
            except Exception as audio_err:
                print("⚠️ Corrupted audio detected — stripping audio.")
                video = video.without_audio()

        # Write video with or without audio
        video.write_videofile(
            output_file_path,
            codec='libx264',
            fps=fps,
            audio=has_audio,
            audio_codec='aac' if has_audio else None,
            verbose=True
        )

    except Exception as e:
        print(f"⚠️ Failed to convert {filename}: {e}")

Converting videos:   0%|          | 0/1 [00:00<?, ?video/s]


🎬 Processing: copy_B52269A0-899E-43A0-A841-93F6DBDD7A9F.mov


Converting videos:   0%|          | 0/1 [00:00<?, ?video/s]

   video.fps: 24.0
   video.duration: 1861.82
   video.size: [2560, 1440]
   video.has_audio: True
Moviepy - Building video /home/siamai/data/Penguin/week7/test/copy_B52269A0-899E-43A0-A841-93F6DBDD7A9F.mp4.
MoviePy - Writing audio in copy_B52269A0-899E-43A0-A841-93F6DBDD7A9FTEMP_MPY_wvf_snd.mp3


Converting videos: 100%|██████████| 1/1 [00:13<00:00, 13.50s/video]

MoviePy - Done.
Moviepy - Writing video /home/siamai/data/Penguin/week7/test/copy_B52269A0-899E-43A0-A841-93F6DBDD7A9F.mp4

⚠️ Failed to convert copy_B52269A0-899E-43A0-A841-93F6DBDD7A9F.mov: must be real number, not NoneType


In [2]:
video_dir = Path('/home/siamai/data/Penguin/week7/test')

for video_file in video_dir.glob("*.mp4"):
    cap = cv2.VideoCapture(str(video_file))
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(f"{video_file.name}: FPS = {fps}")
    cap.release()


s06_smiley.mp4: FPS = 25.0


In [3]:
VIDEO_DIR = Path("/home/siamai/data/Penguin/week7/test")         
OUTPUT_DIR = Path("/home/siamai/data/Penguin/week7/json1")   
  
FRAME_INTERVAL = 1         
USE_VISIBILITY = True   
NUM_WORKERS = mp.cpu_count() - 1 if mp.cpu_count() > 1 else 1  
CHUNKSIZE = 2               


In [4]:
def extract_pose_keypoints(video_path_str, output_path_str):
    video_path = Path(video_path_str)
    output_path = Path(output_path_str)

    mp_pose = mp_pose_module.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False)

    cap = cv2.VideoCapture(str(video_path))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    results = []
    frame_idx = 0
    success = True
    

    with tqdm(total=total_frames, desc=f"Processing {video_path.name}", unit="frame", position=1, leave=False) as pbar:
        while success:
            success, frame = cap.read()
            if not success:
                break

            if frame_idx % FRAME_INTERVAL == 0:
                image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                result = pose.process(image_rgb)

                if result.pose_landmarks:
                    keypoints = []
                    for lm in result.pose_landmarks.landmark:
                        if USE_VISIBILITY:
                            keypoints.append([lm.x, lm.y, lm.z, lm.visibility])
                        else:
                            keypoints.append([lm.x, lm.y, lm.z])
                    results.append({
                        "frame": frame_idx,
                        "keypoints": keypoints
                    })

            frame_idx += 1
            pbar.update(1)

    pose.close()
    cap.release()

    with open(output_path, 'w') as f:
        json.dump(results, f, indent=2)

    return f"{video_path.name} done."
def process_wrapper(args):
    return extract_pose_keypoints(*args)

def batch_process_all_videos(video_dir: Path, output_dir: Path):
    video_dir = Path(video_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)

    video_files = list(video_dir.glob("*.mp4")) + list(video_dir.glob("*.avi")) + list(video_dir.glob("*.mov"))
    tasks = [(str(vf), str(output_dir / f"{vf.stem}_keypoints.json")) for vf in video_files]

    print(f"🚀 Starting batch with {len(tasks)} video(s) using {NUM_WORKERS} process(es)...\n")

    with mp.Pool(NUM_WORKERS) as pool:
        results = list(tqdm(
            pool.imap_unordered(process_wrapper, tasks, chunksize=CHUNKSIZE),
            total=len(tasks),
            desc=" Batch",
            position=0
        ))

    print("\n All videos processed:")
    for r in results:
        print(r)



In [7]:
batch_process_all_videos(VIDEO_DIR, OUTPUT_DIR)


🚀 Starting batch with 5 video(s) using 11 process(es)...



📦 Batch:   0%|          | 0/5 [00:00<?, ?it/s]WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
I0000 00:00:1750080267.412037  890134 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1750080267.412010  890135 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1750080267.412768  890133 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1750080267.569051  890184 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 575.57.08), renderer: NVIDIA H100 80GB HBM3/PCIe/SSE2
I0000 00:00:1750080267.569288  890185 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 575.57.08), renderer: NVIDIA H100 80GB HBM3/PCIe/SSE2
I0000 00:00:1750080267.569533  890183 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 575.57.08), renderer: NVIDIA H100 80GB HBM3/PCIe/SSE2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK


✨ All videos processed:
✅ s03_smiley.mp4 done.
✅ s01_smiley.mp4 done.
✅ s05_smiley.mp4 done.
✅ s04_smiley.mp4 done.
✅ s02_smiley.mp4 done.


In [2]:
json_path = "/home/siamai/data/chuniji/Arealweek7/json"

In [3]:
print(json_path)

/home/siamai/data/chuniji/Arealweek7/json


In [4]:
import pandas as pd
import json

def timestamp_to_seconds(t):
    m = int(t[1:3])
    s = int(t[3:5])
    return m * 60 + s

def load_keypoints(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    # Convert list to dict: frame_idx -> keypoints
    return {item["frame"]: item["keypoints"] for item in data}

def get_keypoint_sequence_around(frame_dict, center_frame, window=15):
    sequence = []
    for f in range(center_frame - window, center_frame + window + 1):
        if f in frame_dict:
            sequence.append(frame_dict[f])
        else:
            sequence.append(None)  # padding if missing
    return sequence

csv_path = "/home/siamai/data/week7/train.csv"
keypoint_dir = Path("/home/siamai/data/chuniji/Arealweek7/json")
fps = 25
window = 15  # = 1 วินาที

df = pd.read_csv(csv_path)
dataset = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    # โค้ดเดิมของคุณ
    filename = row['Filename'].replace(".mp4", "")
    timestamp = row['Time']
    label = row[2:].values.astype(int).tolist()

    sec = timestamp_to_seconds(timestamp)
    center_frame = int(sec * fps)
    
    json_path = keypoint_dir / f"{filename}_keypoints.json"
    if not json_path.exists():
        continue

    keypoint_dict = load_keypoints(json_path)
    sequence = get_keypoint_sequence_around(keypoint_dict, center_frame, window=window)

    dataset.append({
        "video": filename,
        "timestamp": timestamp,
        "center_frame": center_frame,
        "sequence": sequence,
        "label": label
    })

output_path = "/home/siamai/data/chuniji/Arealweek7/dataset2.json"
with open(output_path, "w") as f:
    json.dump(dataset, f)
print(f"บันทึก dataset ลงไฟล์ {output_path} เรียบร้อย")


100%|██████████| 371/371 [26:27<00:00,  4.28s/it]


บันทึก dataset ลงไฟล์ /home/siamai/data/chuniji/Arealweek7/dataset2.json เรียบร้อย


In [ ]:
# import numpy as np
# import json
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# # ระบุตำแหน่งไฟล์ของคุณ
# file_path = "/home/siamai/data/chuniji/Arealweek7/dataset.json"

# try:
#     with open(file_path) as f:
#         data = json.load(f)
# except FileNotFoundError:
#     print(f"⛔️ Error: The file was not found at {file_path}")
#     file_path = "dataset.json"
#     try:
#         with open(file_path) as f:
#             data = json.load(f)
#     except FileNotFoundError:
#         print(f"⛔️ Error: Also could not find {file_path}. Please check the file path.")
#         data = []

# X_seq = []
# y_seq = []

# for i, sample in enumerate(data):
#     sequence = []
#     last_valid_frame = None

#     for j, keypoints_per_frame in enumerate(sample["sequence"]):
#         kp_array = np.array(keypoints_per_frame)

#         if kp_array.ndim == 2 and kp_array.shape[0] == 33 and kp_array.shape[1] >= 2:
#             processed_frame = kp_array[:, :2].flatten()
#             sequence.append(processed_frame)
#             last_valid_frame = processed_frame
#         else:
#             if last_valid_frame is not None:
#                 sequence.append(last_valid_frame)

#     if len(sequence) == 0:
#         print(f"⛔ Skipped sample {i} because it contains NO valid frames to begin with.")
#         continue

#     X_seq.append(np.array(sequence))
#     y_seq.append(sample["label"])

# if X_seq:
#     X_seq_padded = pad_sequences(X_seq, dtype='float32', padding='post')

#     X_seq = np.array(X_seq_padded)
#     y_seq = np.array(y_seq)

#     print("✅ Data processing complete.")
#     print("✅ X_seq shape:", X_seq.shape)
#     print("✅ y_seq shape:", y_seq.shape)

#     # --- ส่วนที่เพิ่มเข้ามา: การบันทึกข้อมูลเป็นไฟล์ .npz ---
#     output_filename = 'processed_dataset.npz'
#     np.savez_compressed(output_filename, X=X_seq, y=y_seq)
#     print(f"✅ Data saved successfully to '{output_filename}'")
#     # -----------------------------------------------------------

# else:
#     print("⛔ No valid data could be processed from the JSON file.")

⛔ Skipped sample 50 because it contains NO valid frames to begin with.
⛔ Skipped sample 51 because it contains NO valid frames to begin with.
⛔ Skipped sample 52 because it contains NO valid frames to begin with.
⛔ Skipped sample 53 because it contains NO valid frames to begin with.
⛔ Skipped sample 54 because it contains NO valid frames to begin with.
⛔ Skipped sample 55 because it contains NO valid frames to begin with.
⛔ Skipped sample 56 because it contains NO valid frames to begin with.
⛔ Skipped sample 57 because it contains NO valid frames to begin with.
⛔ Skipped sample 58 because it contains NO valid frames to begin with.
⛔ Skipped sample 59 because it contains NO valid frames to begin with.
⛔ Skipped sample 60 because it contains NO valid frames to begin with.
⛔ Skipped sample 61 because it contains NO valid frames to begin with.
⛔ Skipped sample 62 because it contains NO valid frames to begin with.
⛔ Skipped sample 63 because it contains NO valid frames to begin with.
⛔ Skip

In [4]:
import numpy as np
import json
from tensorflow.keras.preprocessing.sequence import pad_sequences

# ระบุตำแหน่งไฟล์ของคุณ
file_path = "/home/siamai/data/Penguin/week7/json/s06_smiley_keypoints.json"

try:
    with open(file_path) as f:
        data = json.load(f)
except FileNotFoundError:
    print(f"⛔️ Error: The file was not found at {file_path}")
    file_path = "dataset.json"
    try:
        with open(file_path) as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"⛔️ Error: Also could not find {file_path}. Please check the file path.")
        data = []

X_seq = []
y_seq = []
# เพิ่มลิสต์สำหรับเก็บข้อมูลเพิ่มเติม
video_names = []
timestamps = []
center_frames = []

for i, sample in enumerate(data):
    sequence = []
    last_valid_frame = None

    for j, keypoints_per_frame in enumerate(sample["sequence"]):
        kp_array = np.array(keypoints_per_frame)

        if kp_array.ndim == 2 and kp_array.shape[0] == 33 and kp_array.shape[1] >= 2:
            processed_frame = kp_array[:, :2].flatten()
            sequence.append(processed_frame)
            last_valid_frame = processed_frame
        else:
            if last_valid_frame is not None:
                sequence.append(last_valid_frame)

    if len(sequence) == 0:
        print(f"⛔ Skipped sample {i} because it contains NO valid frames to begin with.")
        continue

    X_seq.append(np.array(sequence))
    y_seq.append(sample["label"])
    # เพิ่มข้อมูลเพิ่มเติมสำหรับแต่ละ sample
    video_names.append(sample["video"])
    timestamps.append(sample["timestamp"])
    center_frames.append(sample["center_frame"])

if X_seq:
    X_seq_padded = pad_sequences(X_seq, dtype='float32', padding='post')

    X_seq = np.array(X_seq_padded)
    y_seq = np.array(y_seq)
    # แปลงลิสต์ข้อมูลเพิ่มเติมให้เป็น NumPy array
    video_names_np = np.array(video_names)
    timestamps_np = np.array(timestamps)
    center_frames_np = np.array(center_frames)


    print("✅ Data processing complete.")
    print("✅ X_seq shape:", X_seq.shape)
    print("✅ y_seq shape:", y_seq.shape)
    print("✅ video_names shape:", video_names_np.shape)
    print("✅ timestamps shape:", timestamps_np.shape)
    print("✅ center_frames shape:", center_frames_np.shape)

    # --- ส่วนที่เพิ่มเข้ามา: การบันทึกข้อมูลเป็นไฟล์ .npz ---
    output_filename = 'processed_datasetd_with_info1.npz'
    np.savez_compressed(output_filename, X=X_seq, y=y_seq,
                         video_names=video_names_np,
                         timestamps=timestamps_np,
                         center_frames=center_frames_np)
    print(f"✅ Data saved successfully to '{output_filename}'")
    # -----------------------------------------------------------

else:
    print("⛔ No valid data could be processed from the JSON file.")

KeyError: 'sequence'

In [6]:
import numpy as np

data = np.load("/home/siamai/data/chuniji/Arealweek7/processed_datasetd_with_info.npz")
X_seq = data['X']  # shape (N, T, 66)
y_seq = data['y']  # shape (N, num_labels)

print(X_seq.shape, y_seq.shape)


(336, 31, 66) (336, 12)


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import StandardScaler

# ==============================================================================
# 1. Configuration / Hyperparameters (ปรับแก้ค่าต่างๆ ที่นี่)
# ==============================================================================
DATA_PATH = "/home/siamai/data/chuniji/Arealweek7/processed_datasetd_with_info.npz"
INPUT_DIM = 66      # จำนวน features ของข้อมูล (เช่น 33 landmarks * 2 มิติ)
HIDDEN_DIM = 256    # ขนาดของ Hidden State ใน LSTM
NUM_LAYERS = 3      # จำนวนชั้นของ LSTM ที่ซ้อนกัน
DROPOUT_P = 0.4     # อัตรา Dropout (เพิ่มขึ้นเล็กน้อยเพื่อป้องกัน Overfitting จากโมเดลที่ซับซ้อนขึ้น)
LEARNING_RATE = 0.001 # เพิ่ม Learning rate เริ่มต้น
BATCH_SIZE = 32
NUM_EPOCHS = 100    # เพิ่มจำนวน Epochs เพื่อให้โมเดลมีเวลาเรียนรู้มากขึ้น
WEIGHT_DECAY = 1e-4 # เพิ่ม L2 Regularization (Weight Decay) เพื่อลด Overfitting
GRAD_CLIP_NORM = 1.0 # เพิ่ม Gradient Clipping เพื่อป้องกัน Exploding Gradients

# ==============================================================================
# 2. Device Configuration
# ==============================================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ==============================================================================
# 3. Data Loading and Preprocessing (Normalization)
# ==============================================================================
# โหลดข้อมูล numpy ดิบ
data = np.load(DATA_PATH)
X_seq_raw = data['X']  # (N, T, 66)
y_seq_raw = data['y']  # (N, num_labels) - สันนิษฐานว่าเป็น one-hot

# แปลง y จาก one-hot เป็น class indices (e.g., [0,0,1,0] -> 2)
# ซึ่งจำเป็นสำหรับ CrossEntropyLoss
y_indices = np.argmax(y_seq_raw, axis=1)
OUTPUT_DIM = y_seq_raw.shape[1] # จำนวน Class ทั้งหมด

# แบ่ง train-val (80:20)
num_samples = X_seq_raw.shape[0]
split_idx = int(num_samples * 0.8)

X_train_np, X_val_np = X_seq_raw[:split_idx], X_seq_raw[split_idx:]
y_train_np, y_val_np = y_indices[:split_idx], y_indices[split_idx:]

# --- ขั้นตอน Normalization ที่เพิ่มเข้ามา (สำคัญมาก) ---
# Reshape ข้อมูล X_train เพื่อ fit scaler: (num_samples * timesteps, num_features)
n_samples_train, n_steps_train, n_features_train = X_train_np.shape
X_train_reshaped = X_train_np.reshape(-1, n_features_train)

# สร้างและ fit scaler กับ "ข้อมูล Train เท่านั้น"
scaler = StandardScaler()
scaler.fit(X_train_reshaped)

# Transform ทั้ง train และ val data ด้วย scaler ตัวเดียวกัน
X_train_scaled = scaler.transform(X_train_reshaped).reshape(n_samples_train, n_steps_train, n_features_train)

# Reshape ข้อมูล validation เพื่อ transform
n_samples_val, n_steps_val, n_features_val = X_val_np.shape
X_val_reshaped = X_val_np.reshape(-1, n_features_val)
X_val_scaled = scaler.transform(X_val_reshaped).reshape(n_samples_val, n_steps_val, n_features_val)
print("Data normalized successfully.")

# ==============================================================================
# 4. Create Tensors and Dataloaders
# ==============================================================================
# แปลง numpy ที่ scale แล้ว -> tensor
X_train = torch.tensor(X_train_scaled, dtype=torch.float32)
X_val = torch.tensor(X_val_scaled, dtype=torch.float32)

# y ต้องเป็น LongTensor สำหรับ CrossEntropyLoss
y_train = torch.tensor(y_train_np, dtype=torch.long)
y_val = torch.tensor(y_val_np, dtype=torch.long)

# สร้าง dataloader
train_ds = TensorDataset(X_train, y_train)
val_ds = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
print(f"DataLoaders created. Train samples: {len(X_train)}, Val samples: {len(X_val)}")


# ==============================================================================
# 5. Model Definition (ปรับปรุงใหม่)
# ==============================================================================
class GestureLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout):
        super().__init__()
        # ใช้ bidirectional=True เพื่อให้โมเดลเรียนรู้จาก 2 ทิศทาง
        self.lstm = nn.LSTM(
            input_dim,
            hidden_dim,
            num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=True
        )
        # hidden_dim ต้องคูณ 2 เพราะ bidirectional LSTM ให้ output 2 เท่า
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        # ไม่ต้องใช้ Sigmoid แล้ว เพราะ CrossEntropyLoss จัดการให้

    def forward(self, x):
        # out shape: (batch, seq_len, hidden_dim * 2)
        out, _ = self.lstm(x)
        # เอาเฉพาะ output ของเฟรมสุดท้าย
        out = out[:, -1, :]
        # ผ่าน linear layer ได้เป็น logits ดิบๆ
        out = self.fc(out)
        return out

# ==============================================================================
# 6. Training Setup
# ==============================================================================
model = GestureLSTM(
    input_dim=INPUT_DIM,
    hidden_dim=HIDDEN_DIM,
    output_dim=OUTPUT_DIM,
    num_layers=NUM_LAYERS,
    dropout=DROPOUT_P
).to(device)

# เปลี่ยน Loss Function เป็น CrossEntropyLoss
# เพิ่ม weight_decay สำหรับ L2 Regularization ใน Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
# ReduceLROnPlateau Scheduler: ลด Learning Rate เมื่อ Validation Loss ไม่ลดลง
# สามารถปรับค่า 'patience' (จำนวน epoch ที่รอ) หรือ 'factor' (อัตราส่วนการลด LR) ได้หากจำเป็น
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)

print("Model and training components initialized.")
print(model)

# ==============================================================================
# 7. Training Loop
# ==============================================================================
for epoch in range(NUM_EPOCHS):
    model.train()
    train_loss = 0
    train_steps = 0
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} - Training")
    for xb, yb in loop:
        xb, yb = xb.to(device), yb.to(device)

        # Forward pass
        preds = model(xb)
        loss = criterion(preds, yb)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        # เพิ่ม Gradient Clipping เพื่อป้องกัน Exploding Gradients
        nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP_NORM)
        optimizer.step()

        train_loss += loss.item()
        train_steps += 1
        loop.set_postfix(loss=loss.item())

    avg_train_loss = train_loss / train_steps

    # --- Validation ---
    model.eval()
    val_loss = 0
    val_steps = 0
    correct_preds = 0
    total_preds = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb)
            loss = criterion(preds, yb)
            val_loss += loss.item()
            val_steps += 1

            # Calculate accuracy
            _, predicted_labels = torch.max(preds, 1)
            total_preds += yb.size(0)
            correct_preds += (predicted_labels == yb).sum().item()

    avg_val_loss = val_loss / val_steps
    val_accuracy = (correct_preds / total_preds) * 100

    # Scheduler step
    scheduler.step(avg_val_loss)

    print(
        f"Epoch {epoch+1}/{NUM_EPOCHS} | "
        f"Train loss: {avg_train_loss:.4f} | "
        f"Val loss: {avg_val_loss:.4f} | "
        f"Val Acc: {val_accuracy:.2f}% | "
        f"LR: {optimizer.param_groups[0]['lr']:.6f}"
    )

print("Training finished.")

# สามารถเพิ่มโค้ดสำหรับบันทึกโมเดลได้ที่นี่
torch.save(model.state_dict(), 'gesture_model_final.pth')
print("Model saved to gesture_model_final.pth")

Using device: cuda
Data normalized successfully.
DataLoaders created. Train samples: 268, Val samples: 68
Model and training components initialized.
GestureLSTM(
  (lstm): LSTM(66, 256, num_layers=3, batch_first=True, dropout=0.4, bidirectional=True)
  (fc): Linear(in_features=512, out_features=12, bias=True)
)


Epoch 1/100 - Training: 100%|██████████| 9/9 [00:00<00:00, 131.35it/s, loss=1.31]


Epoch 1/100 | Train loss: 2.0983 | Val loss: 1.8737 | Val Acc: 20.59% | LR: 0.001000


Epoch 2/100 - Training: 100%|██████████| 9/9 [00:00<00:00, 133.36it/s, loss=1.42]


Epoch 2/100 | Train loss: 1.7343 | Val loss: 1.6191 | Val Acc: 22.06% | LR: 0.001000


...

Epoch 98/100 | Train loss: 0.6488 | Val loss: 1.9799 | Val Acc: 50.00% | LR: 0.000000


Epoch 99/100 - Training: 100%|██████████| 9/9 [00:00<00:00, 133.78it/s, loss=0.969]


Epoch 99/100 | Train loss: 0.6635 | Val loss: 1.9799 | Val Acc: 50.00% | LR: 0.000000


Epoch 100/100 - Training: 100%|██████████| 9/9 [00:00<00:00, 133.43it/s, loss=0.902]


Epoch 100/100 | Train loss: 0.6735 | Val loss: 1.9800 | Val Acc: 50.00% | LR: 0.000000
Training finished.
Model saved to gesture_model_final.pth


In [2]:
import torch
import numpy as np

# Load preprocessed data
data = np.load("processed_dataset7.npz", allow_pickle=True)
X = data["X"]
y = data["y"]
filenames = data["filenames"]

# Convert to torch tensor
X_tensor = torch.tensor(X, dtype=torch.float32)

# Load your PyTorch model
model = torch.load("/home/siamai/data/chuniji/Arealweek7/dgesture_model_final.pth")  # Replace with your model file path
model.eval()

# Run inference (no gradient needed)
with torch.no_grad():
    outputs = model(X_tensor)
    predictions = torch.sigmoid(outputs)  # Use sigmoid if multi-label
    predicted_labels = (predictions > 0.5).int().numpy()

# Show some results
for i in range(5):  # First 5 samples
    print(f"🎥 Sample: {filenames[i]}")
    print(f"✅ Ground Truth: {y[i]}")
    print(f"🔮 Prediction  : {predicted_labels[i]}")
    print("--------")


AttributeError: 'collections.OrderedDict' object has no attribute 'eval'